# Data Cleaning Listado de Listado de Beneficiarios 2019-2022

## Autores
- José Luis Delgado Dávara
- Arturo Ortiz Aguilar
- Beltrán Valle Gutiérrez-Cortines

## Importante leer para entender

En este Notebook se trabaja con 3 listados importantes:

1. Beneficiarios2019_2022 -> Dataset con el listado de TODOS los beneficiarios.
2. Estados_Beneficiarios_2019_2022 -> Dataset sólo con los estados *únicos* encontrados en el dataset anterior.
3. Diccionario -> Emparejamiento entre ambos listados de beneficiarios
4. Dataset_Inegi -> Catálogo obtenido de Inegi

In [1]:
import pandas as pd
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re
import unidecode

In [2]:
def load_datasets(directory):
    # Get a list of all CSV files in the directory
    csv_files = glob.glob(os.path.join(directory, '*.csv'))

    # Read each CSV file and store the DataFrame in a list
    dataframes = [pd.read_csv(file, encoding='utf-8', index_col=0, skiprows=0) for file in csv_files]

    # Print the number of rows for each DataFrame
    for i, df in enumerate(dataframes):
        print(f"Number of rows in DataFrame {i+1}: {df.shape[0]}")

    # Calculate the sum of rows in each individual dataset
    individual_row_sum = sum([df.shape[0] for df in dataframes])

    # Concatenate all DataFrames in the list
    merged_df = pd.concat(dataframes, join='inner', ignore_index=True)

    return merged_df, individual_row_sum


def clean_text(text):
    """
    De esta manera tenemos el texto sin espacios blancos extra y sobre todo con todas las palabras con capitalización correcta.
    """
    if pd.isna(text):
        return text
    text = text.strip()  # Eliminate white spaces
    text = text.lower()  # Convert to lowercase
    text = unidecode.unidecode(text)  # Remove accents
    text = re.sub('-.*-', '', text) #Remove what is in between - -
    text = re.sub('\s+', ' ', text)  # Eliminate extra white spaces
    text = re.sub('^\s+|\s+?$', '', text)  # Eliminate spaces at the beginning and end
    return text

# 1. Lectura de los datos

### Lectura del dataset del INEGI

In [3]:
path_dataset_inegi = '../../data/dataset_inegi.csv'
dataset_inegi = pd.read_csv(path_dataset_inegi, encoding='cp1252', dtype={'CVE_ENT': str, 'CVE_MUN': str, 'CVE_LOC': str})

### Lectura del listado de Beneficiarios 2019-2022

In [4]:
listado_beneficiarios, sumOfRows = load_datasets("../../data/productores_beneficiarios 2019-2022")

Number of rows in DataFrame 1: 826385
Number of rows in DataFrame 2: 394986
Number of rows in DataFrame 3: 369124
Number of rows in DataFrame 4: 278550


In [5]:
nan_rows = listado_beneficiarios.isna()

In [6]:
sumOfRows

1869045

In [7]:
listado_beneficiarios.head()

,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,FECHA,MONTO FEDERAL,APOYO,ACTIVIDAD,ESLABÓN,CICLO AGRÍCOLA
0,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,PUEBLO VIEJO,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,2022-06-26,12244.18,Fertilizantes,AGRARIA,AGRARIA,PV2022
1,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,PUEBLO VIEJO,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,2022-06-26,6122.09,Fertilizantes,AGRARIA,AGRARIA,PV2022
2,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,AGUA ZARCA,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,2022-07-12,12244.18,Fertilizantes,AGRARIA,AGRARIA,PV2022
3,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,AGUA ZARCA,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,2022-07-12,12244.18,Fertilizantes,AGRARIA,AGRARIA,PV2022
4,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,YERBASANTITA,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,2022-06-20,12244.18,Fertilizantes,AGRARIA,AGRARIA,PV2022


In [8]:
listado_beneficiarios.columns

Index(['ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'ESTRATIFICACIÓN',
       'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE', 'PRODUCTO', 'FECHA',
       'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN', 'CICLO AGRÍCOLA'],
      dtype='object')

# 2. Limpieza de los datos

### 2.1 INEGI

In [9]:
# Revisamos las columnas del dataset
dataset_inegi.columns

Index(['MAPA', 'Estatus', 'CVE_ENT', 'NOM_ENT', 'NOM_ABR', 'CVE_MUN',
       'NOM_MUN', 'CVE_LOC', 'NOM_LOC', 'AMBITO', 'LATITUD', 'LONGITUD',
       'LAT_DECIMAL', 'LON_DECIMAL', 'ALTITUD', 'CVE_CARTA', 'POB_TOTAL',
       'POB_MASCULINA', 'POB_FEMENINA', 'TOTAL DE VIVIENDAS HABITADAS'],
      dtype='object')

In [10]:
# Eliminamos las columnas que no son de interés
COLUMNS_TO_DROP = ['MAPA', 'Estatus', 'NOM_ABR', 'AMBITO', 'LATITUD', 'LONGITUD',
                   'LAT_DECIMAL', 'LON_DECIMAL', 'ALTITUD', 'CVE_CARTA', 'POB_TOTAL',
                   'POB_MASCULINA', 'POB_FEMENINA', 'TOTAL DE VIVIENDAS HABITADAS']
dataset_inegi = dataset_inegi.drop(COLUMNS_TO_DROP, axis=1)

In [11]:
# Renombrar las columnas
dataset_inegi.rename(columns={
    'CVE_ENT': 'CVE_ENT',
    'NOM_ENT': 'Entidad_inegi',
    'CVE_MUN': 'CVE_MUN',
    'NOM_MUN': 'Municipio_inegi',
    'CVE_LOC': 'CVE_LOC',
    'NOM_LOC': 'Localidad_inegi',
}, inplace=True)

#dataset_inegi['CVE_LOC'] = dataset_inegi['CVE_LOC'].astype(str)
# Las claves de entidad y municipio serán tratadas numéricamente en la limpieza aunque posteriormente se les asignará el tipo de cadena de texto para tener el estándar.
dataset_inegi.dtypes

CVE_ENT            object
Entidad_inegi      object
CVE_MUN            object
Municipio_inegi    object
CVE_LOC            object
Localidad_inegi    object
dtype: object

In [12]:
number_of_duplicates = dataset_inegi.duplicated().sum()
print(f"Number of duplicates: {number_of_duplicates}")

Number of duplicates: 0


In [13]:
# Revisamos la cantidad de filas y columnas del dataset
print("Shape of dataset_inegi: ", dataset_inegi.shape)

dataset_inegi_uniquemun = dataset_inegi.drop_duplicates(subset=['CVE_MUN', 'Municipio_inegi'], keep='first')
print("Shape of dataset_inegi_uniquemun: ", dataset_inegi_uniquemun.shape)

Shape of dataset_inegi:  (299568, 6)
Shape of dataset_inegi_uniquemun:  (2467, 6)


In [14]:
dataset_inegi_uniquemun

,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,CVE_LOC,Localidad_inegi
0,01,Aguascalientes,001,Aguascalientes,0001,Aguascalientes
708,01,Aguascalientes,002,Asientos,0001,Asientos
945,01,Aguascalientes,003,Calvillo,0001,Calvillo
1237,01,Aguascalientes,004,Cosío,0001,Cosío
1330,01,Aguascalientes,005,Jesús María,0001,Jesús María
...,...,...,...,...,...,...
299150,32,Zacatecas,054,Villa Hidalgo,0001,Villa Hidalgo
299211,32,Zacatecas,055,Villanueva,0001,Villanueva
299363,32,Zacatecas,056,Zacatecas,0001,Zacatecas
299484,32,Zacatecas,057,Trancoso,0001,Trancoso


In [15]:
# Revisamos la cantidad de filas y columnas del dataset
print("Shape of dataset_inegi: ", dataset_inegi.shape)

dataset_inegi_loc = dataset_inegi.drop_duplicates(subset=['CVE_LOC', 'Localidad_inegi'], keep='first')
print("Shape of dataset_inegi_uniqueloc: ", dataset_inegi_loc.shape)

Shape of dataset_inegi:  (299568, 6)
Shape of dataset_inegi_uniqueloc:  (285783, 6)


In [16]:
# Group by 'Localidad_inegi' and aggregate unique 'Municipio_inegi' values
localidad_municipio = dataset_inegi.groupby('Localidad_inegi')['Municipio_inegi'].unique()

# Create a summary string for each 'localidad_inegi' showing count and municipio names
localidad_municipio_summary = localidad_municipio.apply(lambda x: f"{len(x)} - {', '.join(x)}")

# Filter to find 'localidad_inegi' with more than one 'municipio_inegi'
multiple_municipios_summary = localidad_municipio_summary[localidad_municipio.apply(len) > 1]

print("Localidad_inegi with multiple municipio_inegi values and their counts:")
print(multiple_municipios_summary)

Localidad_inegi with multiple municipio_inegi values and their counts:
Localidad_inegi
10 de Abril        8 - Mapastepec, Suchiate, San Luis de la Paz, ...
10 de Mayo                                     2 - Carmen, Iliatenco
12 de Diciembre         2 - Venustiano Carranza, Santiago Miahuatlán
12 de Julio                     2 - La Trinitaria, San Juan Mazatlán
12 de Octubre      4 - Altamirano, Ocozocoautla de Espinosa, Saba...
                                         ...                        
Ánimas                               2 - Ensenada, Guadalupe y Calvo
Ávila [Rancho]                             2 - Ensenada, San Quintín
Órganos                                     2 - Mezquital, Fresnillo
Último Esfuerzo               3 - Caborca, Balancán, Emiliano Zapata
Úrsulo Galván      24 - Ensenada, Palenque, Salto de Agua, Solosu...
Name: Municipio_inegi, Length: 18494, dtype: object


Por aquí existen localidades con nombres de fechas lo cual debe ser común en México ya que se repiten en diversas ocasiones.

In [17]:
# Filter the dataset for rows where 'Municipio' equals '10 De Abril'
filtered_dataset = dataset_inegi[dataset_inegi['Localidad_inegi'] == '10 de Abril']

# Display the filtered dataset
print(filtered_dataset)

       CVE_ENT Entidad_inegi CVE_MUN     Municipio_inegi CVE_LOC  \
42725       07       Chiapas     051          Mapastepec    0278   
52029       07       Chiapas     087            Suchiate    0293   
104096      11    Guanajuato     033  San Luis de la Paz    0835   
106159      12      Guerrero     001  Acapulco de Juárez    0377   
159080      17       Morelos     031           Zacatepec    0029   
176504      20        Oaxaca     044         Loma Bonita    0221   
191848      21        Puebla     106            Ocoyucan    0031   
229683      26        Sonora     033          Huatabampo    0107   

       Localidad_inegi  
42725      10 de Abril  
52029      10 de Abril  
104096     10 de Abril  
106159     10 de Abril  
159080     10 de Abril  
176504     10 de Abril  
191848     10 de Abril  
229683     10 de Abril  


A partir de aquí seguimos trabajando con el listado de Estados y Municipios limpio de Inegi (sin repetir) "dataset_inegi_clean".

In [18]:
# Revisamos las primeras filas del dataset con las columnas seleccionadas
dataset_inegi_uniquemun.head()

,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,CVE_LOC,Localidad_inegi
0,01,Aguascalientes,001,Aguascalientes,0001,Aguascalientes
708,01,Aguascalientes,002,Asientos,0001,Asientos
945,01,Aguascalientes,003,Calvillo,0001,Calvillo
1237,01,Aguascalientes,004,Cosío,0001,Cosío
1330,01,Aguascalientes,005,Jesús María,0001,Jesús María


In [19]:
dataset_inegi_uniquemun.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2467 entries, 0 to 299526
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CVE_ENT          2467 non-null   object
 1   Entidad_inegi    2467 non-null   object
 2   CVE_MUN          2467 non-null   object
 3   Municipio_inegi  2467 non-null   object
 4   CVE_LOC          2467 non-null   object
 5   Localidad_inegi  2467 non-null   object
dtypes: object(6)
memory usage: 134.9+ KB


In [20]:
print("Los valores únicos en cada columna son:\n", dataset_inegi_uniquemun.nunique())

Los valores únicos en cada columna son:
 CVE_ENT              32
Entidad_inegi        32
CVE_MUN             570
Municipio_inegi    2332
CVE_LOC               3
Localidad_inegi    2365
dtype: int64


In [21]:
print("Los valores únicos en cada columna son:\n", dataset_inegi.nunique())

Los valores únicos en cada columna son:
 CVE_ENT                32
Entidad_inegi          32
CVE_MUN               570
Municipio_inegi      2332
CVE_LOC              5692
Localidad_inegi    149004
dtype: int64


#### 2.1.1 Estandarización de nombre de municipios

Con el fin de poder hacer un merge bajo los mismos nombres, hacemos una limpieza de los datos.

In [22]:
# Estandarizamos la limpieza de los datos
dataset_inegi_uniquemun['Entidad_c_inegi'] = dataset_inegi_uniquemun['Entidad_inegi'].apply(clean_text)
dataset_inegi_uniquemun['Municipio_c_inegi'] = dataset_inegi_uniquemun['Municipio_inegi'].apply(clean_text)

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_89061/4159994235.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_uniquemun['Entidad_c_inegi'] = dataset_inegi_uniquemun['Entidad_inegi'].apply(clean_text)
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_89061/4159994235.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_uniquemun['Municipio_c_inegi'] = dataset_inegi_uniquemun['Municipio_inegi'].apply(clean_text)


In [23]:
# Creamos una columna con la clave única por municipio

dataset_inegi_uniquemun['KEY_inegi'] = dataset_inegi_uniquemun['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_uniquemun[
    'Municipio_c_inegi'].astype(str)

dataset_inegi_uniquemun.head()

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_89061/2100087808.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_uniquemun['KEY_inegi'] = dataset_inegi_uniquemun['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_uniquemun[


,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,CVE_LOC,Localidad_inegi,Entidad_c_inegi,Municipio_c_inegi,KEY_inegi
0,01,Aguascalientes,001,Aguascalientes,0001,Aguascalientes,aguascalientes,aguascalientes,aguascalientes-aguascalientes
708,01,Aguascalientes,002,Asientos,0001,Asientos,aguascalientes,asientos,aguascalientes-asientos
945,01,Aguascalientes,003,Calvillo,0001,Calvillo,aguascalientes,calvillo,aguascalientes-calvillo
1237,01,Aguascalientes,004,Cosío,0001,Cosío,aguascalientes,cosio,aguascalientes-cosio
1330,01,Aguascalientes,005,Jesús María,0001,Jesús María,aguascalientes,jesus maria,aguascalientes-jesus maria


In [24]:
dataset_inegi_uniquemun.tail()

,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,CVE_LOC,Localidad_inegi,Entidad_c_inegi,Municipio_c_inegi,KEY_inegi
299150,32,Zacatecas,054,Villa Hidalgo,0001,Villa Hidalgo,zacatecas,villa hidalgo,zacatecas-villa hidalgo
299211,32,Zacatecas,055,Villanueva,0001,Villanueva,zacatecas,villanueva,zacatecas-villanueva
299363,32,Zacatecas,056,Zacatecas,0001,Zacatecas,zacatecas,zacatecas,zacatecas-zacatecas
299484,32,Zacatecas,057,Trancoso,0001,Trancoso,zacatecas,trancoso,zacatecas-trancoso
299526,32,Zacatecas,058,Santa María de la Paz,0001,Santa María de la Paz,zacatecas,santa maria de la paz,zacatecas-santa maria de la paz


In [25]:
dataset_inegi_uniquemun.drop_duplicates(subset=['KEY_inegi'], keep='first', inplace=True)

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_89061/1014247258.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_uniquemun.drop_duplicates(subset=['KEY_inegi'], keep='first', inplace=True)


In [26]:
dataset_inegi_uniquemun.shape

(2465, 9)

## 2.2 Beneficiarios 2023

### Creación de Estados_Beneficiarios2023
Este dataset es una versión de Beneficiarios2023 pero más ligera y sin repeticiones.

In [27]:
listado_beneficiarios.dtypes

ZONA                object
ENTIDAD             object
MUNICIPIO           object
LOCALIDAD           object
ESTRATIFICACIÓN     object
PROGRAMA            object
COMPONENTE          object
SUBCOMPONENTE       object
PRODUCTO            object
FECHA               object
MONTO FEDERAL      float64
APOYO               object
ACTIVIDAD           object
ESLABÓN             object
CICLO AGRÍCOLA      object
dtype: object

In [28]:
descriptive_stats = listado_beneficiarios.describe(include='all').transpose()

# Mostrar las estadísticas descriptivas
print(descriptive_stats)

                     count unique                           top     freq  \
ZONA               1869033      8                   CENTRO PAÍS  1424972   
ENTIDAD            1869033     10                      GUERRERO  1279305   
MUNICIPIO          1869033   1025            CHILAPA DE ÁLVAREZ    60148   
LOCALIDAD          1869033   9979                     TECOANAPA    10820   
ESTRATIFICACIÓN    1869033      7                          Alto  1253286   
PROGRAMA           1869033      1                 FERTILIZANTES  1869033   
COMPONENTE         1869033      1                 FERTILIZANTES  1869033   
SUBCOMPONENTE      1869033      1                 FERTILIZANTES  1869033   
PRODUCTO           1869033      7  Cultivo de maíz grano blanco  1132720   
FECHA              1869033    815                    15/07/2019    17568   
MONTO FEDERAL    1869033.0    NaN                           NaN      NaN   
APOYO              1869033      1                 Fertilizantes  1869033   
ACTIVIDAD   

In [29]:
rows_with_nan = listado_beneficiarios[listado_beneficiarios.isna().any(axis=1)]

In [30]:
rows_with_nan

,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,FECHA,MONTO FEDERAL,APOYO,ACTIVIDAD,ESLABÓN,CICLO AGRÍCOLA
826382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
826383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
826384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1221368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1221369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1221370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1590492,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1590493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1590494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1869042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
listado_beneficiarios.head()

,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,FECHA,MONTO FEDERAL,APOYO,ACTIVIDAD,ESLABÓN,CICLO AGRÍCOLA
0,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,PUEBLO VIEJO,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,2022-06-26,12244.18,Fertilizantes,AGRARIA,AGRARIA,PV2022
1,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,PUEBLO VIEJO,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,2022-06-26,6122.09,Fertilizantes,AGRARIA,AGRARIA,PV2022
2,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,AGUA ZARCA,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,2022-07-12,12244.18,Fertilizantes,AGRARIA,AGRARIA,PV2022
3,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,AGUA ZARCA,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,2022-07-12,12244.18,Fertilizantes,AGRARIA,AGRARIA,PV2022
4,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,YERBASANTITA,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,2022-06-20,12244.18,Fertilizantes,AGRARIA,AGRARIA,PV2022


In [32]:
Estados_beneficiarios2019_2022 = listado_beneficiarios[['ENTIDAD', 'MUNICIPIO']]

In [33]:
Estados_beneficiarios2019_2022 = Estados_beneficiarios2019_2022.drop_duplicates()

In [34]:
# Estandarizamos la limpieza de los datos
Estados_beneficiarios2019_2022['ESTADO_c_benef'] = Estados_beneficiarios2019_2022['ENTIDAD'].apply(clean_text)
Estados_beneficiarios2019_2022['MUNICIPIO_c_benef'] = Estados_beneficiarios2019_2022['MUNICIPIO'].apply(clean_text)

In [35]:
Estados_beneficiarios2019_2022.shape

(1034, 4)

In [36]:
Estados_beneficiarios2019_2022['KEY_benef'] = Estados_beneficiarios2019_2022['ESTADO_c_benef'].astype(str) + '-' + Estados_beneficiarios2019_2022[
    'MUNICIPIO_c_benef'].astype(str)

Estados_beneficiarios2019_2022.head()


,ENTIDAD,MUNICIPIO,ESTADO_c_benef,MUNICIPIO_c_benef,KEY_benef
0,GUERRERO,COYUCA DE BENÍTEZ,guerrero,coyuca de benitez,guerrero-coyuca de benitez
5,GUERRERO,CUTZAMALA DE PINZÓN,guerrero,cutzamala de pinzon,guerrero-cutzamala de pinzon
6,OAXACA,SAN JUAN GUICHICOVI,oaxaca,san juan guichicovi,oaxaca-san juan guichicovi
7,GUERRERO,ARCELIA,guerrero,arcelia,guerrero-arcelia
8,GUERRERO,TLALCHAPA,guerrero,tlalchapa,guerrero-tlalchapa


In [37]:
# Valores únicos y la cantidad de cada columna
# Obtener estadísticas descriptivas para todas las variables

descriptive_stats = Estados_beneficiarios2019_2022.describe(include='all').transpose()

# Mostrar las estadísticas descriptivas
print(descriptive_stats)

                  count unique                         top freq
ENTIDAD            1033     10                      OAXACA  558
MUNICIPIO          1033   1025             EMILIANO ZAPATA    3
ESTADO_c_benef     1033     10                      oaxaca  558
MUNICIPIO_c_benef  1033   1025             emiliano zapata    3
KEY_benef          1034   1034  guerrero-coyuca de benitez    1


In [38]:
Estados_beneficiarios2019_2022 = Estados_beneficiarios2019_2022.drop_duplicates(subset='KEY_benef')

In [39]:
Estados_beneficiarios2019_2022.shape

(1034, 5)

# 3. Diccionario de los datasets de INEGI Y LISTADO BENEFICIARIOS 2023

El objetivo de esta sección es crear un diccionario de códigos según INEGI para los municipios Listado_beneficiarios2023. Para ello haremos un Left join entre Estados_Beneficiarios2023 y dataset_inegi_clean.

### 3.1 Creamos las columnas clave

Confirmamos que tengamos las columnas clave en abos datasets.

In [40]:
dataset_inegi.head()

,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,CVE_LOC,Localidad_inegi
0,01,Aguascalientes,001,Aguascalientes,0001,Aguascalientes
1,01,Aguascalientes,001,Aguascalientes,0094,Granja Adelita
2,01,Aguascalientes,001,Aguascalientes,0096,Agua Azul
3,01,Aguascalientes,001,Aguascalientes,0100,Rancho Alegre
4,01,Aguascalientes,001,Aguascalientes,0102,Los Arbolitos [Rancho]


In [41]:
Estados_beneficiarios2019_2022.head()

,ENTIDAD,MUNICIPIO,ESTADO_c_benef,MUNICIPIO_c_benef,KEY_benef
0,GUERRERO,COYUCA DE BENÍTEZ,guerrero,coyuca de benitez,guerrero-coyuca de benitez
5,GUERRERO,CUTZAMALA DE PINZÓN,guerrero,cutzamala de pinzon,guerrero-cutzamala de pinzon
6,OAXACA,SAN JUAN GUICHICOVI,oaxaca,san juan guichicovi,oaxaca-san juan guichicovi
7,GUERRERO,ARCELIA,guerrero,arcelia,guerrero-arcelia
8,GUERRERO,TLALCHAPA,guerrero,tlalchapa,guerrero-tlalchapa


In [42]:
dataset_inegi.shape

(299568, 6)

In [43]:
Estados_beneficiarios2019_2022.shape

(1034, 5)

### 3.1 Left join (Para crear diccionario)

Creamos el diccionario.

In [44]:
def fuzzy_merge_benef2019_2022(df_benef, df_inegi, key1, key2, threshold=90, limit=1):
    """
    df_benef: DataFrame de la derecha (el DataFrame con el que se quiere hacer el join)
    df_inegi: DataFrame de la izquierda (el DataFrame principal)
    key1: Columna de la clave en df_benef
    key2: Columna de la clave en df_inegi
    threshold: Umbral de coincidencia difusa
    limit: Número de coincidencias a encontrar
    """
    # Convertir la columna clave de df_benef a una lista
    s = df_benef[key1].tolist()
    
    # Encontrar las mejores coincidencias para cada clave en df_inegi
    matches = df_inegi[key2].apply(lambda x: process.extractOne(x, s, score_cutoff=threshold))

    # Crear columnas temporales en df_inegi con las mejores coincidencias y sus puntuaciones
    df_inegi['best_match'] = [match[0] if match else None for match in matches]
    df_inegi['match_score'] = [match[1] if match else None for match in matches]

    # Hacer el merge con las mejores coincidencias, asegurando que df_inegi sea el DataFrame de la izquierda
    df_merged = pd.merge(df_inegi, df_benef, left_on='best_match', right_on=key1, how='left',
                         suffixes=('_inegi', '_benef'))
    
    return df_merged

In [45]:
diccionario = fuzzy_merge_benef2019_2022(Estados_beneficiarios2019_2022, dataset_inegi_uniquemun, 'KEY_benef', 'KEY_inegi')
# Mostrar el resultado
diccionario.columns

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_89061/934286962.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inegi['best_match'] = [match[0] if match else None for match in matches]
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_89061/934286962.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inegi['match_score'] = [match[1] if match else None for match in matches]


Index(['CVE_ENT', 'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi', 'CVE_LOC',
       'Localidad_inegi', 'Entidad_c_inegi', 'Municipio_c_inegi', 'KEY_inegi',
       'best_match', 'match_score', 'ENTIDAD', 'MUNICIPIO', 'ESTADO_c_benef',
       'MUNICIPIO_c_benef', 'KEY_benef'],
      dtype='object')

In [46]:
diccionario.shape

(2465, 16)

In [47]:
# Valores únicos y la cantidad de cada columna
# Obtener estadísticas descriptivas para todas las variables

descriptive_stats = diccionario.describe(include='all').transpose()

# Mostrar las estadísticas descriptivas
print(descriptive_stats)

                    count unique                            top  freq  \
CVE_ENT              2465     32                             20   568   
Entidad_inegi        2465     32                         Oaxaca   568   
CVE_MUN              2465    569                            003    31   
Municipio_inegi      2465   2332                         Ocampo     6   
CVE_LOC              2465      3                           0001  2463   
Localidad_inegi      2465   2363                  Villa Hidalgo     6   
Entidad_c_inegi      2465     32                         oaxaca   568   
Municipio_c_inegi    2465   2332                         ocampo     6   
KEY_inegi            2465   2465  aguascalientes-aguascalientes     1   
best_match           1139   1028                  puebla-puebla    91   
match_score        1139.0    NaN                            NaN   NaN   
ENTIDAD              1139      9                         OAXACA   558   
MUNICIPIO            1139   1021                   

In [48]:
nan_rows = diccionario.isna()

In [49]:
diccionario.to_csv('../../data/diccionario_benef2019_2022.csv', index=False)

In [50]:
diccionario_benef2019_2022 = pd.read_csv('../../data/diccionario_benef2019_2022.csv')

## 3.2 Listado beneficiarios2023

Esta sección se encarga de completar el listado original de Beneficiarios Autorizados con los nombre corregido de INEGI usando el diccionario.

In [51]:
# Crear una variable KEY en listado de productores y el diccionario para hacer el join

# Clean listado beneficiarios
listado_beneficiarios['ESTADO_Clean'] = listado_beneficiarios['ENTIDAD'].apply(clean_text)
listado_beneficiarios['MUNICIPIO_Clean'] = listado_beneficiarios['MUNICIPIO'].apply(clean_text)

# Create KEY in listado beneficiarios
listado_beneficiarios['Estado-mun-KEY'] = listado_beneficiarios['ESTADO_Clean'].astype(str) + '-' + listado_beneficiarios[
    'MUNICIPIO_Clean'].astype(str)

In [52]:
nan_rows = listado_beneficiarios.isna()

In [53]:
nan_rows.sum()

ZONA               12
ENTIDAD            12
MUNICIPIO          12
LOCALIDAD          12
ESTRATIFICACIÓN    12
PROGRAMA           12
COMPONENTE         12
SUBCOMPONENTE      12
PRODUCTO           12
FECHA              12
MONTO FEDERAL      12
APOYO              12
ACTIVIDAD          12
ESLABÓN            12
CICLO AGRÍCOLA     12
ESTADO_Clean       12
MUNICIPIO_Clean    12
Estado-mun-KEY      0
dtype: int64

In [54]:
print(listado_beneficiarios.shape)
listado_beneficiarios.columns


(1869045, 18)


Index(['ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'ESTRATIFICACIÓN',
       'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE', 'PRODUCTO', 'FECHA',
       'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN', 'CICLO AGRÍCOLA',
       'ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY'],
      dtype='object')

In [55]:
# Lectura del diccionario Verificado Simple
#diccionario_verificado = pd.read_csv('../../data/Diccionario_benef_manual_verificado.csv')
diccionario_verificado_simple = pd.read_csv('../../data/Diccionario_Simple.csv')

In [56]:
descriptive_stats = diccionario_verificado_simple.describe(include='all').transpose()

# Mostrar las estadísticas descriptivas
print(descriptive_stats)

                       count unique                                   top  \
ESTADO                  2362     30                                OAXACA   
MUNICIPIO               2362   2237                       EMILIANO ZAPATA   
ESTADO_c_benef          2362     31                                oaxaca   
MUNICIPIO_c_benef       2362   2237                       emiliano zapata   
KEY_benef               2362   2358  oaxaca-san pedro mixtepec -dto. 26 -   
KEY_benef_Verificado    2362   2334            jalisco-canadas de obregon   
best_match              2345   2316             oaxaca-san pedro mixtepec   
match_score           2345.0    NaN                                   NaN   
CVE_ENT               2362.0    NaN                                   NaN   
Entidad_inegi           2362     30                                Oaxaca   
CVE_MUN               2362.0    NaN                                   NaN   
Municipio_inegi         2362   2215                                Ocampo   

In [57]:
diccionario_verificado_simple.columns

Index(['ESTADO', 'MUNICIPIO', 'ESTADO_c_benef', 'MUNICIPIO_c_benef',
       'KEY_benef', 'KEY_benef_Verificado', 'best_match', 'match_score',
       'CVE_ENT', 'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi',
       'Entidad_c_inegi', 'Municipio_c_inegi', 'KEY_inegi'],
      dtype='object')

In [58]:
diccionario_verificado_simple.rename(columns={'ï»¿KEY_benef': 'KEY_benef'}, inplace=True)

In [59]:
diccionario_simple = pd.read_csv('../../data/Diccionario_Simple.csv')

El diccionatio contiene KEY_benef únicos (Freq = 1), mientras que KEY_benef_Verificado puede repetirse.

### MERGE

Armamos por partes el dataset definitivo:
1. Juntamos Listado_beneficiarios con el diccionario simple. (listado_beneficiario_parte_I)
2. Al df anterior juntamos las claver provenientes del catálogo de Inegi. (listado_beneficiario_parte_II)

In [60]:
# Hacer el join de la Parte I
listado_beneficiarios_parte_I = pd.merge(listado_beneficiarios, diccionario_benef2019_2022, left_on="Estado-mun-KEY",
                                        right_on="KEY_benef_Verificado", how='left', suffixes=('_benef', '_inegi'))

In [61]:
listado_beneficiarios_parte_I.columns

Index(['ZONA', 'ENTIDAD', 'MUNICIPIO_benef', 'LOCALIDAD', 'ESTRATIFICACIÓN',
       'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE', 'PRODUCTO', 'FECHA',
       'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN', 'CICLO AGRÍCOLA',
       'ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY', 'ESTADO',
       'MUNICIPIO_inegi', 'ESTADO_c_benef', 'MUNICIPIO_c_benef', 'KEY_benef',
       'KEY_benef_Verificado', 'best_match', 'match_score', 'CVE_ENT',
       'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi', 'Entidad_c_inegi',
       'Municipio_c_inegi', 'KEY_inegi'],
      dtype='object')

In [62]:
listado_beneficiarios_parte_I_nan_counts = listado_beneficiarios_parte_I.isna().sum()
print(listado_beneficiarios_parte_I_nan_counts)

ZONA                       12
ENTIDAD                    12
MUNICIPIO_benef            12
LOCALIDAD                  12
ESTRATIFICACIÓN            12
PROGRAMA                   12
COMPONENTE                 12
SUBCOMPONENTE              12
PRODUCTO                   12
FECHA                      12
MONTO FEDERAL              12
APOYO                      12
ACTIVIDAD                  12
ESLABÓN                    12
CICLO AGRÍCOLA             12
ESTADO_Clean               12
MUNICIPIO_Clean            12
Estado-mun-KEY              0
ESTADO                  13583
MUNICIPIO_inegi         13583
ESTADO_c_benef          13583
MUNICIPIO_c_benef       13583
KEY_benef               13583
KEY_benef_Verificado    13583
best_match              17909
match_score             17909
CVE_ENT                 13583
Entidad_inegi           13583
CVE_MUN                 13583
Municipio_inegi         13583
Entidad_c_inegi         17909
Municipio_c_inegi       17909
KEY_inegi               17909
dtype: int

In [63]:
# Hacer el join de la Parte II
listado_beneficiarios_parte_II = pd.merge(listado_beneficiarios_parte_I, dataset_inegi_uniquemun, left_on="KEY_benef",
                                        right_on="KEY_inegi", how='left', suffixes=('_benef', '_inegi'))

In [64]:
listado_beneficiarios_parte_II.columns

Index(['ZONA', 'ENTIDAD', 'MUNICIPIO_benef', 'LOCALIDAD', 'ESTRATIFICACIÓN',
       'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE', 'PRODUCTO', 'FECHA',
       'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN', 'CICLO AGRÍCOLA',
       'ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY', 'ESTADO',
       'MUNICIPIO_inegi', 'ESTADO_c_benef', 'MUNICIPIO_c_benef', 'KEY_benef',
       'KEY_benef_Verificado', 'best_match', 'match_score', 'CVE_ENT_benef',
       'Entidad_inegi_benef', 'CVE_MUN_benef', 'Municipio_inegi_benef',
       'Entidad_c_inegi_benef', 'Municipio_c_inegi_benef', 'KEY_inegi_benef',
       'CVE_ENT_inegi', 'Entidad_inegi_inegi', 'CVE_MUN_inegi',
       'Municipio_inegi_inegi', 'CVE_LOC', 'Localidad_inegi',
       'Entidad_c_inegi_inegi', 'Municipio_c_inegi_inegi', 'KEY_inegi_inegi'],
      dtype='object')

In [65]:
listado_beneficiarios_parte_II.shape

(1880396, 42)

In [66]:
# Valores únicos y la cantidad de cada columna
# Obtener estadísticas descriptivas para todas las variables

descriptive_stats = listado_beneficiarios_parte_II.describe(include='all').transpose()

# Mostrar las estadísticas descriptivas
print(descriptive_stats)

                             count unique                           top  \
ZONA                       1880384      8                   CENTRO PAÍS   
ENTIDAD                    1880384     10                      GUERRERO   
MUNICIPIO_benef            1880384   1025            CHILAPA DE ÁLVAREZ   
LOCALIDAD                  1880384   9979                     TECOANAPA   
ESTRATIFICACIÓN            1880384      7                          Alto   
PROGRAMA                   1880384      1                 FERTILIZANTES   
COMPONENTE                 1880384      1                 FERTILIZANTES   
SUBCOMPONENTE              1880384      1                 FERTILIZANTES   
PRODUCTO                   1880384      7  Cultivo de maíz grano blanco   
FECHA                      1880384    815                    15/07/2019   
MONTO FEDERAL            1880384.0    NaN                           NaN   
APOYO                      1880384      1                 Fertilizantes   
ACTIVIDAD                

In [67]:
listado_beneficiarios_parte_II = listado_beneficiarios_parte_II.drop_duplicates(subset = ['ACUSE ESTATAL'],keep = 'first')
listado_beneficiarios_parte_II.shape

KeyError: Index(['ACUSE ESTATAL'], dtype='object')

In [ ]:
# Seleccionamos las columnas que nos interesan
listado_beneficiarios_parte_II = listado_beneficiarios_parte_II[
    ['ESTADO', 'MUNICIPIO', 'ACUSE ESTATAL', 'APELLIDO PATERNO', 'APELLIDO MATERNO',
     'NOMBRE (S)', 'PAQUETE', 'KEY_benef_Verificado', 'Entidad_inegi', 'Municipio_inegi', 'CVE_ENT', 'CVE_MUN']]

KeyError: "None of [Index(['ESTADO', 'MUNICIPIO', 'ACUSE ESTATAL', 'APELLIDO PATERNO',\n       'APELLIDO MATERNO', 'NOMBRE (S)', 'PAQUETE', 'KEY_benef_Verificado',\n       'Entidad_inegi', 'Municipio_inegi', 'CVE_ENT', 'CVE_MUN'],\n      dtype='object')] are in the [columns]"

In [ ]:
# Revisamos el dataset
print(listado_beneficiarios_parte_II.shape)
print(listado_beneficiarios_parte_II.columns)
print(listado_beneficiarios_parte_II.head())

(1869045, 43)
Index(['ZONA', 'ENTIDAD_benef', 'MUNICIPIO_benef', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA', 'ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY',
       'ENTIDAD_inegi', 'MUNICIPIO_inegi', 'ESTADO_c_benef',
       'MUNICIPIO_c_benef', 'KEY_benef', 'best_match', 'match_score',
       'CVE_ENT_benef', 'Entidad_inegi_benef', 'CVE_MUN_benef',
       'Municipio_inegi_benef', 'CVE_LOC_benef', 'Localidad_inegi_benef',
       'Entidad_c_inegi_benef', 'Municipio_c_inegi_benef', 'KEY_inegi_benef',
       'CVE_ENT_inegi', 'Entidad_inegi_inegi', 'CVE_MUN_inegi',
       'Municipio_inegi_inegi', 'CVE_LOC_inegi', 'Localidad_inegi_inegi',
       'Entidad_c_inegi_inegi', 'Municipio_c_inegi_inegi', 'KEY_inegi_inegi'],
      dtype='object')
          ZONA ENTIDAD_benef    MUNICIPIO_benef     LOCALIDAD ESTRATIFICACIÓN  \
0  CENTRO PAÍS      GUERRERO  

In [ ]:
nan_counts = listado_beneficiarios_parte_II.isna().sum()
print(nan_counts)

ZONA                          12
ENTIDAD_benef                 12
MUNICIPIO_benef               12
LOCALIDAD                     12
ESTRATIFICACIÓN               12
PROGRAMA                      12
COMPONENTE                    12
SUBCOMPONENTE                 12
PRODUCTO                      12
FECHA                         12
MONTO FEDERAL                 12
APOYO                         12
ACTIVIDAD                     12
ESLABÓN                       12
CICLO AGRÍCOLA                12
ESTADO_Clean                  12
MUNICIPIO_Clean               12
Estado-mun-KEY                 0
ENTIDAD_inegi                 12
MUNICIPIO_inegi               12
ESTADO_c_benef                12
MUNICIPIO_c_benef             12
KEY_benef                      0
best_match                  5680
match_score                 5680
CVE_ENT_benef               5680
Entidad_inegi_benef         5680
CVE_MUN_benef               5680
Municipio_inegi_benef       5680
CVE_LOC_benef               5680
Localidad_

Hay 0 filas que no han sido emparejadas.

In [ ]:
listado_beneficiarios_parte_II.to_csv('../../data/LISTADO_BENEFICIARIOS2023_COMPLETO.csv', index=False)

KeyboardInterrupt: 

In [ ]:
listado_beneficiarios_parte_II.shape


(1679795, 12)